In [ ]:
import findspark
findspark.init()

import folium

from folium.plugins import MarkerCluster
from pyspark import sql, SparkConf, SparkContext
from ipywidgets import interact

In [ ]:
conf = SparkConf().setAppName("Read_CSV")
sc = SparkContext(conf=conf)
sqlContext = sql.SQLContext(sc)

dfCities = sqlContext.read.csv("Steden.csv", header=True)
dfPollution = sqlContext.read.csv("Luchtvervuiling.csv", header=True)

coordinates = (20.593684, 78.96288)
pMap = folium.Map(location=coordinates, zoom_start=4)

In [ ]:
TYPES_P = ["so2", "no2", "rspm", "spm"]
HIGH_P = [10, 18, 200, 300]
LOW_P = [5, 10, 80, 200]

In [ ]:
STRONG_BLUE = "#3186cc"
RED = "#ff0000"
GREEN = "#7CFC00"
LIGHT_ORANGE = "#ffd27f"

In [ ]:
dfMapCities = dfCities.select("lat", "lng", "city")
dfMapPollution = dfPollution.select("location", "so2", "no2", "rspm", "spm")
colorPoint = STRONG_BLUE

@interact(types=TYPES_P)
def get_pollution(types):
    index = 0
    joinDF = dfMapPollution.join(dfMapCities, dfMapPollution.location == dfMapCities.city, how="right")
    if types == "so2":
        joinDF = joinDF.filter("so2 != ''").filter("so2 != 'NA'").filter("lat != ''")\
        .filter("lng != ''").dropDuplicates(["city"]).collect()
    elif types == "no2":
        joinDF = joinDF.filter("no2 != ''").filter("no2 != 'NA'").filter("lat != ''")\
        .filter("lng != ''").dropDuplicates(["city"]).collect()
    elif types == "rspm":
        joinDF = joinDF.filter("rspm != ''").filter("rspm != 'NA'").filter("lat != ''")\
        .filter("lng != ''").dropDuplicates(["city"]).collect()
    elif types == "spm":
        joinDF = joinDF.filter("spm != ''").filter("spm != 'NA'").filter("lat != ''")\
        .filter("lng != ''").dropDuplicates(["city"]).collect()
    
    typeIndex = TYPES_P.index(types)
    highPValue = HIGH_P[typeIndex]
    lowPValue = LOW_P[typeIndex]
    typeIndexOnJoinDF = typeIndex + 1
    
    for line in joinDF:
        pollution = joinDF[index][typeIndexOnJoinDF]
        if float(pollution) > highPValue:
            colorPoint = RED
        elif float(pollution) < lowPValue:
            colorPoint = GREEN
        else:
            colorPoint = LIGHT_ORANGE

        folium.CircleMarker(
            location = [joinDF[index][5], joinDF[index][6]],
            radius = 4,
            popup = pollution,
            color = colorPoint,
            fill = True,
            fill_color = STRONG_BLUE
        ).add_to(pMap)
        index = index + 1
    return pMap